<a href="https://colab.research.google.com/github/codejawk/Sarcasm_detectiopn/blob/master/Sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np

In [2]:
sents=['My name is Jansheir',
       'I love my dog',
       'my dog loves me.',
       'wow! what a wonderful dog.']

token=Tokenizer(num_words=5,oov_token='UNK')
token.fit_on_texts(sents)

sequences=token.texts_to_sequences(sents)
pad_seq=pad_sequences(sequences,maxlen=5,padding='post')

In [3]:
WORD_INDEX=token.word_index



In [4]:
WORD_INDEX

{'UNK': 1,
 'a': 13,
 'dog': 3,
 'i': 7,
 'is': 5,
 'jansheir': 6,
 'love': 8,
 'loves': 9,
 'me': 10,
 'my': 2,
 'name': 4,
 'what': 12,
 'wonderful': 14,
 'wow': 11}

In [5]:
import json
file_name="/content/Sarcasm_Headlines_Dataset.json"
data=[]
with open(file_name) as f:
  for line in f:
    data.append(json.loads(line))
        

In [6]:
data[2]

{'article_link': 'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
 'headline': "mom starting to fear son's web series closest thing she will have to grandchild",
 'is_sarcastic': 1}

In [7]:
sentences = [] 
labels = []
urls = []
for item in data:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])



In [8]:
print(len(sentences),len(labels),len(urls))

26709 26709 26709


In [38]:
#splitting the data
train_sent=sentences[0:20000]
train_labels=labels[0:20000]
test_sent=sentences[20001:]
test_labels=labels[20001:]


### converting train labels to nparray for special purpose
train_labels=np.array(train_labels)

test_labels=np.array(test_labels)

In [39]:
token=Tokenizer(num_words=10000,oov_token='UNK')
token.fit_on_texts(train_sent)
word_index=token.word_index
train_seq=token.texts_to_sequences(train_sent)
train_pad=pad_sequences(train_seq,padding='post',truncating='post',maxlen=120)


test_seq=token.texts_to_sequences(test_sent)
test_pad=pad_sequences(test_seq,padding='post',truncating='post',maxlen=120)

In [11]:
print(train_sent[1])
train_pad[1]

the 'roseanne' revival catches up to our thorny political mood, for better and worse


array([   4, 6840, 3096, 3097,   23,    2,  161,    1,  390, 2842,    6,
        251,    9,  889,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [26]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_index(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

decode_index(train_pad[1])

"the 'roseanne' revival catches up to our UNK political mood for better and worse ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?"

In [25]:
print(reverse_word_index.get(188,'?'))
print(word_index.get('4'))

4
188


In [40]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=120),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [41]:
num_epochs = 10
model.fit(train_pad, train_labels, epochs=num_epochs, validation_data=(test_pad, test_labels))

Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6083 - accuracy: 0.6424 - val_loss: 0.4695 - val_accuracy: 0.8268
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3834 - accuracy: 0.8793 - val_loss: 0.3919 - val_accuracy: 0.8473
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2695 - accuracy: 0.9291 - val_loss: 0.3515 - val_accuracy: 0.8640
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1523 - accuracy: 0.9536 - val_loss: 0.3376 - val_accuracy: 0.8630
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0973 - accuracy: 0.9718 - val_loss: 0.3730 - val_accuracy: 0.8608
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.0643 - accuracy: 0.9829 - val_loss: 0.4179 - val_accuracy: 0.8584
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.0419 - accuracy: 0.9909 - val_loss: 0.4609 - val_accuracy: 0.8579
Epoch 

In [34]:
type(train_pad)

numpy.ndarray